In [25]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select


In [11]:
url='https://weteach.edu.tw/my/'

options = Options()
options.headless = False
driverPath = r'C:\Users\user\Documents\GitHub\moodleCrawl\geckodriver.exe'
driver = webdriver.Firefox(options=options, executable_path=driverPath)
#driver = webdriver.Firefox(executable_path=driverPath)

#headless模式
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
#driver = webdriver.Chrome(executable_path='chromedriver.exe')

driver.set_page_load_timeout(60)
driver.get(url)


In [ ]:
#登入moodle
username = 'username'
password = 'password'
element = driver.find_element_by_id('login_username')
element.send_keys(username)
element = driver.find_element_by_id('login_password')
element.send_keys(password)

driver.find_element_by_css_selector(".btn-primary").click()


In [ ]:
#進入課程網址

In [30]:
#取得測驗的連結
id = 1735
#https://weteach.edu.tw/course/view.php?id=1735
courseurl = 'http://weteach.edu.tw/course/view.php?'
quizname=[]
quizurl=[]
for i in range(6):
    url = courseurl + 'id='+str(id) +'&section='+str(i+1)
    driver.get(url)
    #取得各section原始碼
    htmltext = driver.page_source

    soup=BeautifulSoup(htmltext,"html.parser")
    
    #搜尋所有含quiz的連結
    links = soup.find_all('a',{'href':re.compile('quiz')})
    for link in links:

        #取得href屬性前先檢查是否存在
        if 'href' in link.attrs:
            href = link['href']
            print(link.text)
            print(href)
            quizname.append(link.text)
            quizurl.append(href)
        else:
            href = None

複式顯微鏡與解剖顯微鏡的比較 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43504
解剖顯微鏡構造填圖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43490
解剖顯微鏡的使用 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43491
複式顯微鏡的使用 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43494
複式顯微鏡的倍率亮度與視野問題 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43505
顯微鏡的使用 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43506
玻片標本製作 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43510
生命現象(1) 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43513
生命現象(2) 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43514
動植物細胞填圖 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43515
細胞構造的功能和名稱配對 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43516
細胞構造的名稱和功能配對 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43517
細胞的構造與生命現象(1) 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43518
細胞的構造與生命現象(2) 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43519
物質進出細胞的練習 測驗卷
https://weteach.edu.tw/mod/quiz/view.php?id=43522
物質進出細胞(1) 測驗卷
https://weteach

In [31]:
#從url找id
quizid=[]
for quiz in quizurl:
    k = re.search(r'id=(.*)$', quiz)
    quizid.append(k.group(1))
quizid

['43504',
 '43490',
 '43491',
 '43494',
 '43505',
 '43506',
 '43510',
 '43513',
 '43514',
 '43515',
 '43516',
 '43517',
 '43518',
 '43519',
 '43522',
 '43523',
 '43524',
 '43530',
 '43531',
 '43533',
 '43537',
 '43539',
 '43541']

In [32]:
#進入試卷編修網址
                        #https://weteach.edu.tw/course/modedit.php?update=43489&return=0&sr=0
quizurlBase = 'https://weteach.edu.tw/course/modedit.php?update='
n = 0



gradecat = '2074'
#2074:A1 2065:A2 2067:A3 2072:B1 2070:B2  2069:B3
gradepass = 85  #及格分數
attempts = 4    #可測驗次數
grademethod = '1' 
#1:最高分數 2:平均分數 3:第一次作答  4:最後一次作答
for i in quizid:
    quizurl = quizurlBase+str(i)+'&return=1'
    
    driver.get(quizurl)

    #展開全部
    driver.find_element_by_css_selector('.collapseexpand').click()

    #修改及格分數
    element =  driver.find_element_by_id('id_gradepass')
    element.clear()
    element.send_keys(gradepass)

    
    s1 = Select(driver.find_element_by_id('id_gradecat'))
    s1.select_by_value(gradecat)
   
    
    #修改允許作答次數
    #搜尋作答次數的選單並點擊
    element = driver.find_element_by_id('id_attempts')
    element.click()
    element.send_keys(attempts)
    
    
    #修改評分方式
    s1 = Select(driver.find_element_by_id('id_grademethod'))
    s1.select_by_value(grademethod)
    
    
    #解開完成進度的選項
    driver.find_element_by_id('id_unlockcompletion').click()


    #儲存並顯示
    driver.find_element_by_id('id_submitbutton2').click()
    print('done:' + quizname[n])
    print(quizurl)
    n = n+1
    print(str(n)+"/"+str(len(quizid)))
    print('========')


done:複式顯微鏡與解剖顯微鏡的比較 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43504&return=1
1/23
done:解剖顯微鏡構造填圖 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43490&return=1
2/23
done:解剖顯微鏡的使用 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43491&return=1
3/23
done:複式顯微鏡的使用 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43494&return=1
4/23
done:複式顯微鏡的倍率亮度與視野問題 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43505&return=1
5/23
done:顯微鏡的使用 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43506&return=1
6/23
done:玻片標本製作 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43510&return=1
7/23
done:生命現象(1) 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43513&return=1
8/23
done:生命現象(2) 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43514&return=1
9/23
done:動植物細胞填圖 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43515&return=1
10/23
done:細胞構造的功能和名稱配對 測驗卷
https://weteach.edu.tw/course/modedit.php?update=43516&return=1
11/23
done:細胞構造的名稱和功能配對 測驗卷
https://

## 以下是測試區

In [28]:
#處理單一連結的測試區

#進入試卷編修網址
url = 'https://weteach.edu.tw/course/modedit.php?update=43504'
driver.get(url)

#element = WebDriverWait(driver,3).until(expected_conditions.presence_of_element_located((By.ID,'id_gradepass')))
#element = driver.find_element_by_name('gradepass')

#展開全部
driver.find_element_by_css_selector('.collapseexpand').click()

#修改及格分數
element =  driver.find_element_by_id('id_gradepass')
element.clear()
element.send_keys('100')

#修改允許作答次數
#搜尋作答次數的選單並點擊
element = driver.find_element_by_id('id_attempts')
element.click()
element.send_keys('7')

#for option in driver.find_elements_by_tag_name('option'):
    #if option.text == '3':
    #    option.click()

grademethod = '2'
#修改評分方式
s1 = Select(driver.find_element_by_id('id_grademethod'))
s1.select_by_value(grademethod)

#element = driver.find_element_by_id('id_grademethod')
#element.click()
#element.send_keys( grademethod )

#解開完成進度的選項
driver.find_element_by_id('id_unlockcompletion').click()


#儲存並顯示
driver.find_element_by_id('id_submitbutton2').click()
print('done')
#element = driver.find_element_by_xpath('//*[@id="id_gradepass"]')


#finally:
    #driver.quit()


done
